# Natural Langauge Processing of ISIS Content
This notebook contains information regarding the efforts of **Kamal Kamalaldin** and **Will Fitzgerald** in filtering ISIS contents that violate the terms of service from the Internet Archive (www.archive.org).

## Requirements
1. **Python 3+** (current version is 3.5). This should be a straightforward thing to [install](https://www.python.org/downloads/)
2. [**pip3**](https://pip.pypa.io/en/stable/installing/) or [**homebrew**](http://brew.sh/) command line tools (homeb. We will be using these tools in install other, more essential tools.
3. **The Internet Archive command tools**, refered to throughout this notebook as *ia*. Here we use pip to install *ia*. It can also be installed using any of the previous command line tools one or more of which we just installed. Because of some issues with using imported modules while using jupyter (IPython notebook), **if you plan on using this notebook to run all the code**, we need to specify that we want to install the *ia* tools in the directory in which jupyter can find the module. Follow the pip3 execurable command before continuing. 
    - using homebrew: "brew install internetarchive"
    - using pip3: "pip3 install internetarchive"
4. [** Vowpal Wabbit**](http://hunch.net/~vw/). For macs, this can be installed using homebrew. For windows, perhaps [scoop](http://scoop.sh/) will work.

In [ ]:
! python3 -V

In [ ]:
! EDIT THE FILEPATH AND DELETE THIS LINE BEFORE EXECUTING
! DONT EXECUTE UNTIL EDITING PATH IN CAPS pip3 install internetarchive -t /PATH/TO/CURRENT/ANACONDA/ENVIRONMENT/lib/python3.5/site-packages/internetarchive

In [ ]:
import internetarchive as ia

## Data collection
At the beginning of our endeavour, we attempted to categories arabic item entries into those violating the term of service, and those who do not. Items can violate the term of service if they are "deemed offensive, disturbing, pornographic, racist, sexist, bizarre, misleading, fraudulent, or otherwise objectionable" (Terms of Service, Internet Archive). As Kamal was the only person knowledgable enough in the Arabic language to understand and categorize items, it was quickly aparent that the rate at which items were categorized was too slow to attain the sufficient data to train a model. Therefore, another approach was considered.

For the data portion, a director tree was created.

In [ ]:
! mkdir data

##### ISIS Collection

A collection of ~5,000 items that included ISIS/ISIL or their arabic counterparts in them was collected. This collection would represent items that were ISIS related, and assumed to be malicious. 

First, items with "الدولة الاسلامية", the arabic equivalent of Islamic State are searched for. We first examine the number of items returned, then we download them.

In [118]:
! mkdir data/ISIS
! echo "Number of matching searches found : "
! ia search "الدولة الاسلامية" -i -n 
! echo "Donwloading item identifiers..."
! ia search "الدولة الاسلامية" -i > data/ISIS/IDList.txt
! echo "Number of item identifiers downloaded:"
! wc -l data/ISIS/IDList.txt

mkdir: data/ISIS: File exists
Number of matching searches found : 
1047
Donwloading item identifiers...
Number of item identifiers downloaded:
    1047 data/ISIS/IDList.txt


If you encounter an error running these commands, ensure that you have the latest version of *ia* installed (See **Requirements** above). 

The -i tag specifies that we are only interested in the ID of the item, not the whole file (we will download those later). The -n tag specifies that we only want to know the number of search results for now.
In the third line we ask for the ID to be downloaded and put in a file called ISIS.txt in the data directory. The head command allows us to examine the first three lines of the file we jsut created.

We now extract the IDs from the file and download the metadata for each item. The files are downloaded for future inspection and cashing. **This is currently a time consuming procedure**

In [119]:
! mkdir data/ISIS/metadata

mkdir: data/ISIS/metadata: File exists


In [121]:
import json

def getIDsFromFile(file):
    IDs = []
    print("Openning ID file...")
    print("Reading IDs from file...")
    for line in ISIS_ID_file:
        IDs.append(line.rstrip())
    print("Finished reading IDs from file. Closing file...")
    ISIS_ID_file.close()
    print("ID File closed!")
    return IDs

def downloadMetaFiles(IDs):
    
    print("Downloading " + str(len(IDs)) + " item metadatas...")
    count = 1
    for ID in IDs:
        file = open("data/ISIS/metadata/"+ID+".txt", 'w')
        meta = ia.get_item(ID).metadata
        json.dump(meta, file)
        if(count % 31 == 0):
            print('downloaded ' + str(count) + ' metadata files so far!')
        count += 1
    print("Successefully downloaded " + str(len(IDs)) + " metadatas!")


ISIS_ID_file = open('data/ISIS/IDList.txt', 'r')
IDs = getIDsFromFile(ISIS_ID_file)
downloadMetaFiles(IDs)

Openning ID file...
Reading IDs from file...
Finished reading IDs from file. Closing file...
ID File closed!
downloaded 31 metadata files so far!
downloaded 62 metadata files so far!
downloaded 93 metadata files so far!
downloaded 124 metadata files so far!
downloaded 155 metadata files so far!
downloaded 186 metadata files so far!
downloaded 217 metadata files so far!
downloaded 248 metadata files so far!
downloaded 279 metadata files so far!
downloaded 310 metadata files so far!
downloaded 341 metadata files so far!
downloaded 372 metadata files so far!
downloaded 403 metadata files so far!
downloaded 434 metadata files so far!
downloaded 465 metadata files so far!
downloaded 496 metadata files so far!
downloaded 527 metadata files so far!
downloaded 558 metadata files so far!
downloaded 589 metadata files so far!
downloaded 620 metadata files so far!
downloaded 651 metadata files so far!
downloaded 682 metadata files so far!
downloaded 713 metadata files so far!
downloaded 744 metad

We now have the metadata readily available in files, and I/O should take a much faster time to retreive the data rather than downloading them again if need be. We define the methods below to extract the metadata back from the files.

In [130]:
import os

def getMetaFromFile(fileDir):
    file = open(fileDir, 'r')
    return json.load(file)
    
def readMetaTextInDirectory(directory):
    return [getMetaFromFile(directory + os.sep + f) 
            for f in os.listdir(directory)
            if  f.endswith('.txt')]

metadata = readMetaTextInDirectory('data/ISIS/metadata')
len(metadata)

1000

This list of metadata will be the actual data points which we will process, develop our model with, and test against!
We need to get it in VW format first. To see what that looks like, skim through the [input format page](https://github.com/JohnLangford/vowpal_wabbit/wiki/Input-format). In a nutshell, each data point (metadata entry) must be on its own line. Each line must begin with a +1 or -1 to signify if it's a positive or negative example (ISIS or not ISIS, respectively, in this case), followed by a pipe "|" that separated each namespace and its features. The format for namespace and features is: "namespac1e: feature1 |namespace2: feature2 |namespeace3: feature3...\n". "|" and ":" are reserved for VW, so we must distinguish their occurance in the metadata from their occrance in the VW format. We replace "|" with "PIPE" and ":" with "COLON" 

In [126]:
def proccessText(text):
    return text.replace(':', 'COLON').replace('|', 'PIPE').replace('\n', ' ').replace('\r', ' ').replace("@", ' ')

def metadataToVWline(metadata: dict, positive: bool):
    ignored_keys = ['mediatype', 'sound', 'color', 'curation']
    data = ''
    for key in metadata:
        if key in ignored_keys:
            continue
        else:
            if(type(metadata[key]) == list):
                string = ' '.join(metadata[key])
                data += key + ': ' + proccessText(string) + " |"
            else:
                data += key + ': ' + proccessText(metadata[key]) + " |"
    #remove last trailing pipe and add new line
    data = data.rstrip('|')
    data += '\n'
    
    if(positive):
        data = "+1 | " + data
    else:
        data = "-1 | " + data
        
    return data
#     ID = metadata.get('identifier', 'NONE').replace(':', 'COLON').replace('|', 'PIPE')
#     uploader = metadata.get('uploader', 'NONE').replace(':', 'COLON').replace('|', 'PIPE')
#     title = metadata.get('title', 'NONE').replace(':', 'COLON').replace('|', 'PIPE')
#     subject = metadata.get('subject', 'NONE').replace(':', 'COLON').replace('|', 'PIPE')
#     description = metadata.get('description', 'NONE').replace(':', 'COLON').replace('|', 'PIPE')
#     publishDate = metadata.get('publishdate', 'NONE').replace(':', 'COLON').replace('|', 'PIPE')
#     addedDate = metadata.get('addeddate', 'NONE').replace(':', 'COLON').replace('|', 'PIPE')
#     collection = metadata.get('collection', 'NONE').replace(':', 'COLON').replace('|', 'PIPE')
#     ID = metadata['identifier']
#     uploader = metadata['uploader']
#     title = metadata['title']
#     subject = metadata['subject']
#     description = metadata['description']
#     publishDate = metadata['publicdate']
#     addedDate = metadata['addeddate']
#     collection = metadata['collection']
#     print(metadata)
    
#     data = "identifier: ".join(ID) + " | uploader: ".join(uploader) + " | title: ".join(title) + " | subject: "
#     data = data + ''.join(subject) + " | description".join(description) + " | publishDate: ".join(publishDate)
#     data = data + " | addedDate: ".join(addedDate) + " | collection: ".join(collection) + "\n"


def writeVWlinesToFile(vwLines, file):
    print("opening Vwneg.txt file to write metadata")
    print("Writing lines to file...")
    for line in vwLines:
        file.write(line)
    print("Closing file...")
    file.close()
    print("File closed!")


vwLines = []
for meta in metadata:
    vwLines.append(metadataToVWline(meta, True))
    
ISIS_Metadata_vw_file = open('data/ISIS/VWneg.txt', 'w')
writeVWlinesToFile(vwLines, ISIS_Metadata_vw_file)

opening Vwneg.txt file to write metadata
Writing lines to Vwneg.txt ...
Closing Vwneg.txt file...
File Vwneg.txt closed!


A sample (~40 items) of this collection was inspected to determine the accuracy of the assumption (how many items in the sample are *actually* ISIS content that violated terms of service). The accurasy of our assumption was recorded as 


$AccViol = \frac{v(s)}{l(s)}$

where $v(s)$ is the number of items violating the terms of service in the sample $s$ and $l(s)$ is the number of items in $s$

In [144]:
import random
random.shuffle(metadata)
AccViol = 27/30
# metadata[:10]


[{'addeddate': '2015-05-06 21:58:34',
  'collection': ['opensource_movies', 'loggedin'],
  'curation': '[curator]validator@archive.org[/curator][date]20150506215900[/date][comment]checked for malware[/comment]',
  'description': '<div>ولاية الرقة\xa0</div><div><br /></div><div>الدولة الاسلامية\xa0</div><div><br /></div>',
  'identifier': 'oma_222_mail_030_201505',
  'mediatype': 'movies',
  'publicdate': '2015-05-06 21:58:34',
  'scanner': 'Internet Archive HTML5 Uploader 1.6.1',
  'subject': 'ولاية الرقة   الدولة الاسلامية',
  'title': '  قصف الطيران النصيري على بيوت المسلمين في ولاية الرقة',
  'uploader': 'oma_222@mail.ru'},
 {'addeddate': '2015-06-25 02:59:10',
  'collection': 'opensource_movies',
  'curation': '[curator]validator@archive.org[/curator][date]20150625025938[/date][comment]checked for malware[/comment]',
  'description': '<div>مؤسسة الاعتصام\xa0</div><div>الدولة الاسلامية</div><div><br /></div><div><br /></div><div><br /></div><div><br /></div><div>\xa0</div>',
  'iden

Main points:
- get negative data
- run it all through VW
- get a proccess to get new data (IDs + metadata)

##### Arabic-non-ISIS Collection

Another collectino of ~5,000 items that included general arabic words was collected. This collection would represent items that included arabic text but were not violating of the terms of service.

In [ ]:
print("Placeholder Code")

A sample (~40 items) of this collection was inspected to determine the accuracy of the assumption (how many items in the sample are *actually* general arabic content that do not violate the terms of service). The accuracy of our assumption was recorded as $AccAra$

In [ ]:
print("Placeholder code")

AccAra = 0